In [1]:
import matplotlib.pyplot as plt
import xtrack as xt
import scipy as sc
import bpmeth
import sympy as sp
import math
import numpy as np
from cpymad.madx import Madx
from sympy.abc import alpha, beta  # Cool notebook greek letters!
import sys
import warnings
import nafflib
from HeLICS_FMA import calc_FMA


  ++++++++++++++++++++++++++++++++++++++++++++
  +     MAD-X 5.09.03  (64 bit, Linux)       +
  + Support: mad@cern.ch, http://cern.ch/mad +
  + Release   date: 2024.04.25               +
  + Execution date: 2026.01.16 16:39:08      +
  ++++++++++++++++++++++++++++++++++++++++++++
! ==== DRIFTS =====

q_drift        = 1.05;

mb_drift       = 0.2;

mb_quad_drift  = 0.15; // space mb2q + endcoils Q

sx_drift       = 0.5;

ss_drift       =  4.;



! ==== DIPOLE GEOMETRY ====

edge          = 30./180.*pi;

ein1          := edge;  

eout1         := edge;



phi            = 60./180.*pi;

rho            = 2.737;

mb_len        := phi*rho;



! ==== QUADRUPOLE STRENGTHS =====

quad_len       = 0.2;

qb_len         = 0.400;

qf_len         = 0.400;

qd_len         = 0.235;

sx_len         = 0.115;





mb_k1          = -0.11; 

qb_k1          = 2.8067743015574944;

qf_k1          = 2.7486028442707378;!2.6444898800172743;    !Qx=2.67, QY=0.7:  2.7486028442707378, 

qd_k1          = -2.6146643

++++++ Error: seterrorflag : Errorcode: 1   Reported from pro_twiss:
++++++ Error: seterrorflag : Description: TWISS failed


In [5]:
##################
# HeLICS lattice #
##################

mad=Madx()
#mad.call("/home/silke/Documents/HeLICS/HeLICS.str")
#mad.call("/home/silke/Documents/HeLICS/HeLICS.seq")
mad.call("/home/silke/Documents/PhD/repos/bpmeth/examples/HeLICS/ring/HeLICS.str")
mad.call("/home/silke/Documents/PhD/repos/bpmeth/examples/HeLICS/ring/HeLICS.seq")
mad.beam()
mad.use("he_ring")

def match_tunes(madx,qx,qy):
    madx.input(f'''
    use, sequence=he_ring;
    MATCH;
    GLOBAL, Q2={qy},Q1={qx};
    vary, name=qd_k1, step=0.0000001;
    vary, name=qf_k1, step=0.0000001;
    simplex, calls=5000, tolerance=1E-10;
    endmatch;''')
    qf = madx.eval('qf_k1')
    qd = madx.eval('qd_k1')
    return qf,qd

def match_disp(madx):
    madx.input(f'''
    use, sequence=he_ring;
    MATCH;
    constraint, sequence=he_ring, range = "qf2b", DX=0.0;
    vary, name=qb_k1, step=0.0000001;
    jacobian, calls=5000, tolerance=1E-10;
    endmatch;''')
    qb = madx.eval('qb_k1') 
    return qb

print("Matched to Qx 2.4, Qy 1.2 with quadrupole strengths qf, qd = ", match_tunes(mad, 2.4, 1.2))

fringe="linear"

line=xt.Line.from_madx_sequence(mad.sequence.he_ring)
line.configure_bend_model(edge=fringe, core='adaptive', num_multipole_kicks=30)
line.particle_ref=xt.Particles(mass0=0.931494 * 4.001506179127e9, q0=2., energy0=(0.931494 * 4.001506179127 + 0.02)*1e9)  # Energy in eV, top energy
# start_elem = "lssc"
# line.cycle(name_first_element=start_elem, inplace=True)  # Such that dipole is not spanning over end-beginning of lattice


  ++++++++++++++++++++++++++++++++++++++++++++
  +     MAD-X 5.09.03  (64 bit, Linux)       +
  + Support: mad@cern.ch, http://cern.ch/mad +
  + Release   date: 2024.04.25               +
  + Execution date: 2026.01.16 16:40:20      +
  ++++++++++++++++++++++++++++++++++++++++++++
! ==== DRIFTS =====

q_drift        = 1.05;

mb_drift       = 0.2;

mb_quad_drift  = 0.15; // space mb2q + endcoils Q

sx_drift       = 0.5;

ss_drift       =  4.;



! ==== DIPOLE GEOMETRY ====

edge          = 30./180.*pi;

ein1          := edge;  

eout1         := edge;



phi            = 60./180.*pi;

rho            = 2.737;

mb_len        := phi*rho;



! ==== QUADRUPOLE STRENGTHS =====

quad_len       = 0.2;

qb_len         = 0.400;

qf_len         = 0.400;

qd_len         = 0.235;

sx_len         = 0.115;





mb_k1          = -0.11; 

qb_k1          = 2.8067743015574944;

qf_k1          = 2.7486028442707378;!2.6444898800172743;    !Qx=2.67, QY=0.7:  2.7486028442707378, 

qd_k1          = -2.6146643

++++++ Error: seterrorflag : Errorcode: 1   Reported from pro_twiss:
++++++ Error: seterrorflag : Description: TWISS failed


START MATCHING

++++++ warning: TWCLORB:  Longitudinal plane might be unstable. Try change lag with 0.5. More information with the debug flag on.                                                                                                                                                   
number of variables:    2
user given constraints: 2
total constraints:      2

START SIMPLEX:

++++++ warning: TWCLORB:  Longitudinal plane might be unstable. Try change lag with 0.5. More information with the debug flag on.                                                                                                                                                   
++++++ warning: TWCLORB:  Longitudinal plane might be unstable. Try change lag with 0.5. More information with the debug flag on.                                                                                                                                                   
++++++ warning: TWCLORB:  Longitudinal plane might be unst

Converting sequence "he_ring": 100%|██████████| 67/67 [00:00<00:00, 92.27it/s]


In [3]:
calc_FMA(line, fringe="linear", n_turn=3000, nr=5, nphi=3)


15 particles created


/home/silke/Documents/PhD/repos/bpmeth/examples/HeLICS/ring/HeLICS_FMA.py:131: RuntimeWarning: divide by zero encountered in log10
  dQ = np.log10(np.sqrt((tunediff(Qx1, Qx2))**2 + tunediff(Qy1, Qy2)**2))


In [4]:
calc_FMA(line, fringe="full", n_turn=3000, nr=5, nphi=3)

15 particles created
